# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [1]:
%pip install leafmap lonboard

In [2]:
import duckdb
import leafmap

In [3]:
%pip install jupysql duckdb-engine

In [4]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [ ]:
url = "https://storage.googleapis.com/qm2/CASA0025/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://storage.googleapis.com/qm2/CASA0025/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 106MB/s]


Extracting files...


'/content/nyc_data.db.zip'

In [ ]:
%sql duckdb:///nyc_data.db

In [8]:
%%sql

INSTALL spatial;
LOAD spatial;

,Success


1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [ ]:
%%sql

select s.name subway_station_name, s.routes subway_routes
from nyc_subway_stations s
inner join nyc_neighborhoods n
  on st_intersects(s.geom, n.geom)
where lower(n.NAME) like '%little italy%'

,subway_station_name,subway_routes
0,Spring St,6


2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [ ]:
%%sql

select distinct n.name neighborhood_name
from nyc_subway_stations s
inner join nyc_neighborhoods n
  on st_intersects(n.geom, s.geom)
where s.routes like '%6%'

,neighborhood_name
0,Gramercy
1,Murray Hill
2,Soundview
3,Upper East Side
4,Financial District
5,Hunts Point
6,East Harlem
7,Little Italy
8,Yorkville
9,Chinatown


3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [ ]:
%%sql

select sum(popn_total)
from nyc_census_blocks b
inner join nyc_neighborhoods n
  on st_intersects(n.geom, b.geom)
where lower(n.name) like '%battery park%'

,sum(popn_total)
0,17153.0


In [ ]:
%%sql

select cast(sum((st_area(st_intersection(n.geom, b.geom)) / st_area(b.geom)) * b.popn_total) as int) weighted_population
from nyc_census_blocks b
inner join nyc_neighborhoods n
  on st_intersects(n.geom, b.geom)
where lower(n.name) like '%battery park%'

,weighted_population
0,11917


4. **What neighborhood has the highest population density (persons/km2)?**


In [ ]:
%%sql

select n.name neighborhood, round(1000000 * sum(b.popn_total) / mean(st_area(n.geom)), 2)
from nyc_census_blocks b
inner join nyc_neighborhoods n
  on st_intersects(n.geom, b.geom)
group by 1
order by 2 desc
limit 20

,neighborhood,"round(((1000000 * sum(b.popn_total)) / mean(st_area(n.geom))), 2)"
0,North Sutton Area,68435.13
1,East Village,50404.48
2,Chinatown,48825.18
3,Carnegie Hill,48543.73
4,Upper East Side,48524.49
5,Little Italy,46769.35
6,Greenwich Village,42835.92
7,Gramercy,41812.39
8,Morris Heights,40687.92
9,Upper West Side,40152.49


In [ ]:
%%sql

select * --n.name neighborhood, round(1000 * sum(b.popn_total) / mean(st_area(n.geom)), 2)
from nyc_census_blocks b
inner join nyc_neighborhoods n
  on st_contains(n.geom, b.geom)
--group by 1
--order by 2 desc
where n.name = 'Annandale'
limit 20

,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geom,BORONAME_1,NAME,geom_1
0,360850170051009,0,0,0,0,0,0,Staten Island,"[5, 4, 248, 0, 0, 0, 0, 0, 163, 9, 11, 73, 235...",Staten Island,Annandale,"[5, 4, 105, 0, 0, 0, 0, 0, 129, 232, 10, 73, 1..."
1,360850170051010,23,22,0,0,1,0,Staten Island,"[5, 4, 168, 0, 0, 0, 0, 0, 41, 0, 11, 73, 236,...",Staten Island,Annandale,"[5, 4, 105, 0, 0, 0, 0, 0, 129, 232, 10, 73, 1..."
2,360850156031011,92,86,0,0,6,0,Staten Island,"[5, 4, 168, 0, 0, 0, 0, 0, 186, 117, 11, 73, 1...",Staten Island,Annandale,"[5, 4, 105, 0, 0, 0, 0, 0, 129, 232, 10, 73, 1..."
3,360850170051011,123,121,0,0,0,2,Staten Island,"[5, 4, 184, 0, 0, 0, 0, 0, 169, 240, 10, 73, 6...",Staten Island,Annandale,"[5, 4, 105, 0, 0, 0, 0, 0, 129, 232, 10, 73, 1..."
4,360850170051012,21,21,0,0,0,0,Staten Island,"[5, 4, 200, 0, 0, 0, 0, 0, 103, 247, 10, 73, 1...",Staten Island,Annandale,"[5, 4, 105, 0, 0, 0, 0, 0, 129, 232, 10, 73, 1..."
5,360850170051013,49,46,0,0,3,0,Staten Island,"[5, 4, 120, 0, 0, 0, 0, 0, 123, 247, 10, 73, 1...",Staten Island,Annandale,"[5, 4, 105, 0, 0, 0, 0, 0, 129, 232, 10, 73, 1..."
6,360850170051026,107,88,0,0,17,2,Staten Island,"[5, 4, 25, 0, 0, 0, 0, 0, 191, 241, 10, 73, 20...",Staten Island,Annandale,"[5, 4, 105, 0, 0, 0, 0, 0, 129, 232, 10, 73, 1..."
7,360850170051032,108,101,0,5,2,0,Staten Island,"[5, 4, 168, 0, 0, 0, 0, 0, 188, 238, 10, 73, 1...",Staten Island,Annandale,"[5, 4, 105, 0, 0, 0, 0, 0, 129, 232, 10, 73, 1..."
8,360850170051039,47,39,3,0,0,5,Staten Island,"[5, 4, 136, 0, 0, 0, 0, 0, 50, 6, 11, 73, 132,...",Staten Island,Annandale,"[5, 4, 105, 0, 0, 0, 0, 0, 129, 232, 10, 73, 1..."
9,360850170051040,69,55,0,0,10,4,Staten Island,"[5, 4, 89, 0, 0, 0, 0, 0, 219, 10, 11, 73, 2, ...",Staten Island,Annandale,"[5, 4, 105, 0, 0, 0, 0, 0, 129, 232, 10, 73, 1..."


When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [5]:
import pandas as pd

In [6]:
%sql duckdb:///:memory:

In [7]:
%%sql
INSTALL httpfs;
LOAD httpfs;

,Success


In [8]:
%%sql
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [9]:
%%sql

create or replace table full_ships as select * from 'https://storage.googleapis.com/qm2/casa0025_ships.csv'

,Success


In [10]:
%%sql

select count(*) from full_ships limit 1

,count_star()
0,101328


In [11]:
%%sql

select * from full_ships limit 10

,vesselid,vessel_name,vsl_descr,dwt,v_length,draught,sog,date,lat,lon,geom
0,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,5.2,2022-07-25 02:53:29,45.151777,36.513327,POINT (36.5133266666667 45.1517766666667)
1,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.7,2022-07-25 03:09:37,45.146487,36.520780,POINT (36.52078 45.1464866666667)
2,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.7,2022-07-25 03:13:58,45.146218,36.521965,POINT (36.521965 45.1462183333333)
3,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.1,2022-07-25 04:16:06,45.145058,36.522020,POINT (36.52202 45.1450583333333)
4,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.0,2022-07-25 05:20:17,45.144933,36.521848,POINT (36.5218483333333 45.1449333333333)
5,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.0,2022-07-25 06:23:57,45.145050,36.521795,POINT (36.521795 45.14505)
6,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.0,2022-07-25 07:25:25,45.146583,36.519967,POINT (36.5199666666667 45.1465833333333)
7,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.2,2022-07-25 10:36:41,45.146813,36.520895,POINT (36.520895 45.1468133333333)
8,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.0,2022-07-25 12:15:05,45.145980,36.522195,POINT (36.522195 45.14598)
9,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.1,2022-07-25 13:41:53,45.145262,36.522122,POINT (36.5221216666667 45.1452616666667)


In [12]:
%%sql

create or replace table ais as
select vesselid, draught, sog, date, geom from full_ships

,Success


In [13]:
%%sql

create or replace table vinfo as
select distinct vesselid, vessel_name, vsl_descr, dwt, v_length from full_ships

,Success


In [14]:
%%sql

select count(*) from vinfo

,count_star()
0,835


In [20]:
%%sql

select column_name, data_type
from information_schema.columns
where table_name = 'ais'

,column_name,data_type
0,vesselid,BIGINT
1,draught,DOUBLE
2,sog,DOUBLE
3,date,TIMESTAMP
4,geom,VARCHAR


In [22]:
%%sql

create or replace table ais as
select * exclude(geom), st_transform(st_geomfromtext(geom), 'EPSG:4326', 'EPSG:3857') as geom from ais

,Success


In [23]:
%%sql

CREATE INDEX ais_rtree ON ais USING RTREE(geom);

,Success


In [24]:
%%sql

select *, st_astext(geom) from ais limit 5

,vesselid,draught,sog,date,geom,st_astext(geom)
0,350053,3.5,5.2,2022-07-25 02:53:29,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",POINT (5026272.786944948 4371486.169389135)
1,350053,3.5,0.7,2022-07-25 03:09:37,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",POINT (5025683.906838652 4372518.547320337)
2,350053,3.5,0.7,2022-07-25 03:13:58,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",POINT (5025654.036108615 4372682.693492584)
3,350053,3.5,0.1,2022-07-25 04:16:06,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",POINT (5025524.905499295 4372690.312152347)
4,350053,3.5,0.0,2022-07-25 05:20:17,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",POINT (5025510.990562946 4372666.532717071)


In [25]:
%%sql

select count(*) from ais

,count_star()
0,101328


## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.

In [28]:
%%sql

with ordered_vessels as (
  select vesselid, sog, date time_start,
    lead(date) over (
      partition by vesselid
      order by date
      ) time_end,
    geom
  from ais
),

near_vessels as (
  select v1.vesselid v1, v2.vesselid v2,
  greatest(v1.time_start, v2.time_start) overlap_start, least(v1.time_end, v2.time_end) overlap_end
  from ordered_vessels v1
  inner join ordered_vessels v2
    on v1.vesselid < v2.vesselid
    and v1.time_start < v2.time_end and v1.time_end > v2.time_start
    and st_dwithin(v1.geom, v2.geom, 500)
    and v1.sog < 1 and v2.sog < 1
),

lag_times as (
  select v1, v2, overlap_start, overlap_end,
    overlap_start - lag(overlap_end) over (
      partition by v1, v2
      order by overlap_start
      ) gap
  from near_vessels
),

events as (
  select v1, v2, overlap_start, overlap_end,
    sum(case
          when gap is null or gap > interval '0 minutes'
            then 1
          else 0
        end
    ) over (
        partition by v1, v2
        order by overlap_start
      ) event_group
  from lag_times
)

select v1 vessel_1, v2 vessel_2,
  min(overlap_start) start_time,
  max(overlap_end) end_time,
  max(overlap_end) - min(overlap_start) duration
from events
group by v1, v2, event_group
having max(overlap_end) - min(overlap_start) >= interval '2 hours'
order by 1,3

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,vessel_1,vessel_2,start_time,end_time,duration
0,54217,13337216,2022-06-15 20:52:12,2022-06-16 02:38:35,0 days 05:46:23
1,54217,235376,2022-06-16 01:32:34,2022-06-16 05:50:36,0 days 04:18:02
2,54217,13337216,2022-06-16 04:50:35,2022-07-05 18:09:33,19 days 13:18:58
3,54217,235376,2022-06-16 13:02:33,2022-07-20 16:42:59,34 days 03:40:26
4,54217,250424,2022-06-17 19:44:36,2022-07-20 16:42:59,32 days 20:58:23
...,...,...,...,...,...
9123,13410995,13432308,2022-06-03 03:15:06,2022-06-03 16:27:31,0 days 13:12:25
9124,13410995,13462305,2022-06-08 22:13:19,2022-06-09 03:01:44,0 days 04:48:25
9125,13410995,13499237,2022-08-02 01:58:19,2022-08-05 21:14:43,3 days 19:16:24
9126,13410995,13420857,2022-08-21 21:50:10,2022-08-22 01:49:45,0 days 03:59:35
